# Cancelación de eco con filtros LMS y RLS

Supongamos que nos comunicamos por teléfono con otra persona y que dicha persona tiene su telefono conectado a un alto-parlante

Cuando la persona nos conteste, lo que ingresará a su micrófono será su respuesta más posibles repeteciones indeseadas de nuestro mensaje original como muestra el siguiente esquema

<img src="../clases/images/echo-room.png" width="300">

Podemos eliminar el "eco" indeseado con un filtro adaptivo como muestra el siguiente diagrama

<img src="../clases/images/adaptive-echo-canceller.png" width="500">

Detalles del filtro cancelador de eco

- Se usa como entrada la señal enviada
- Se usa como salida deseada la señal recibida (contiene eco)
- El objetivo es que el filtro aprenda los coeficientes del sistema reverberante del primer diagrama
- El error entre la salida y la entrada será la señal recibida limpia
- En general el éco no va a ocurrir al mismo tiempo que el habla. Se debe tener cuidado de adaptar el filtro solo al eco
- El filtro debe tener suficientes coeficientes para poder cancelar el eco, mientras más tiempo alla entre voz y eco más largo debe ser el filtro

Referencia: http://dsp-book.narod.ru/307.pdf

Veamos como se programa paso a paso

In [ ]:
%matplotlib notebook
import librosa
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
from IPython.display import Audio

# Esta será la señal r(t) es decir "nuestra voz transmitida"
# Se usa la frecuencia de muestreo típica de las comunicaciones telefónicas: 8000 Hz

r, Fs = librosa.load("../data/hola2.ogg", sr=8000)
r = np.pad(r, pad_width=(0, 6000))
Audio(r, rate=int(Fs), normalize=False)

A continuación simularemos una sala que introduce reverberación con un filtro FIR

In [ ]:
T = 200
h = np.concatenate((np.zeros(T), 
                    [0.4], np.zeros(T), 
                    [0.3], np.zeros(T), 
                    [0.2]))

# Esta es la voz con eco agregado
#r += np.random.randn(len(r))*0.005
rh = scipy.signal.convolve(r, h, mode='full')[:len(r)]

fig, ax = plt.subplots(figsize=(8, 3), tight_layout=True)
ax.plot(r[:6000], label='r')
ax.plot(rh[:6000], label='r*h')
ax.legend()
Audio(rh, rate=int(Fs), normalize=False)

In [ ]:
# Esta es la respuesta de la persona denotada por s(t)

s, Fs = librosa.load("../data/hola1.ogg", sr=8000)
s = np.pad(s, pad_width=(3000, 0))
Audio(s, rate=int(Fs), normalize=False)

In [ ]:
# Esta es la señal de nuestro amigo que nos llega a nosotros
# Es una mezcla de la voz de nuestro amig@ + el eco nuestro
srh = rh.copy()
srh[:len(s)] += s
fig, ax = plt.subplots(figsize=(8, 3), tight_layout=True)
ax.plot(r, label='r')
ax.plot(srh, label='r*h + s')
ax.legend()
Audio(srh, rate=int(Fs), normalize=False)

In [ ]:
class Filtro_RLS:
    
    def __init__(self, L, beta=0.99, delta=1e-2):
        self.L = L
        self.w = np.zeros(shape=(L+1, ))
        self.beta = beta
        self.delta = delta
        self.Phi_inv = delta*np.eye(L+1)
        
    def update(self, un, dn):
        # Cálculo de la ganancia
        pi = np.dot(un.T, self.Phi_inv)
        kn = pi.T/(self.beta + np.inner(pi, un))
        # Actualizar el vector de pesos
        error = dn - np.dot(self.w, un)
        self.w += kn*error
        # Actualizar el inverso de Phi
        self.Phi_inv = (self.Phi_inv - np.outer(kn, pi))*self.beta**-1
        return np.dot(self.w, un)

- Probar $\beta = [0.999, 0.99, 0.9]$
- Probar $\delta = [1e-3, 1e-2, 1e-1, 1]$

In [ ]:
filt = Filtro_RLS(500, beta=0.999, delta=1e-3)
#filt = Filtro_RLS(500, beta=0.99, delta=1e+1)
hhat = np.zeros(shape=(filt.L+1, len(r)))
rhhat = np.zeros(shape=(len(r)))
for k in range(L+1, len(r)):
    hhat[:, k] = filt.w
    if np.sum(r[k-filt.L-1:k]**2)/(filt.L+1) > 1e-4:
        rhhat[k] = filt.update(r[k-filt.L-1:k][::-1], srh[k])
    else: # El filtro se aplica solo cuando yo hablo
        rhhat[k] = 0.
# La estimación de la voz limpia de nuestro amig@
shat = srh - rhhat

fig, ax = plt.subplots(2, figsize=(7, 4), tight_layout=True, sharex=True)
ax[0].plot(rh, alpha=0.5, label='r(t)*h(t)');
ax[0].plot(rhhat, alpha=0.5, label='r(t)*hhat(t)');
ax[0].legend()
ax[1].plot(shat, alpha=0.75, label='shat(t)');
ax[1].plot(s, alpha=0.75, label='s(t)');
ax[1].legend()
Audio(shat, rate=int(Fs), normalize=False)